### 3.4 Basic building blocks

- > `Q` can have different `seq_len` from `KV`, while `V` can have different `d_model` from `QK`. For former, think cross attenion in translation, where both languages can have different number of tokens.
- > `parameter` do not have `.weights`, use `.data`.
- > each group of `param_groups` has its own state.
- > checkout `_optimize` in "/home/azureuser/02-fun/cs336-assignment1-basics/tests/test_optimizer.py
- > `self.q_mha` did not get moved to new device because it's not `parameter`
- > in-place assignment can cause trouble during back-propagation.

In [ ]:
## `encode` take very long to encode. Multiprocess?
## check output with `decode`

# with open("/home/azureuser/02-fun/cs336-assignment1-basics/data/TinyStoriesV2-GPT4-valid.txt", encoding="utf-8") as f:
#     txt = f.read()
# tokenizer = Tokenizer.from_files(
#     vocab_filepath="/home/azureuser/02-fun/cs336-assignment1-basics/train_bpe_vocab_ts.json",
#     merges_filepath="/home/azureuser/02-fun/cs336-assignment1-basics/train_bpe_merges_ts.txt",
#     special_tokens = ["<|endoftext|>"],
# )
# txt_ids = tokenizer.encode(txt)
# np.save( "../data/TinyStoriesV2-valid.npy", np.array(txt_ids, dtype=np.int32))


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from einops import rearrange, einsum

from model import *
from nn_utils import *
from data import *
from optimizer import *
from tokenizer import *

In [ ]:
class Dataset:
    def __init__(self, src: str):
        self.dataset = np.load(src, mmap_mode = "r")

    def get_batch(self, batch_size, context_length, device="cpu"):
        return get_batch(self.dataset, batch_size, context_length, device)


In [ ]:
dataset = Dataset("../data/TinyStoriesV2-valid.npy")

In [ ]:
batch_size = 4
context_length = 64
device = "cuda:0"
device = "cpu"
batch = dataset.get_batch(batch_size, context_length, device)


In [ ]:
vocab_filepath="/home/azureuser/02-fun/cs336-assignment1-basics/train_bpe_vocab_ts.json"
merges_filepath="/home/azureuser/02-fun/cs336-assignment1-basics/train_bpe_merges_ts.txt"
special_tokens = ["<|endoftext|>"]

vocab, _ = get_tokenizer_from_vocab_merges_path(vocab_filepath, merges_filepath)

In [ ]:
model = transformer_lm(
    d_model =16, d_ff = 64, num_heads = 4, rope_theta = 1e4,
    num_layers=3, vocab_size=len(vocab), context_length = 128
)
model.to(device)

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=0.01,
    betas=(0.9, 0.999),
    eps=1e-8,
)


In [ ]:
def train_one_step(batch):
    # Use 1000 optimization steps for testing
    optimizer.zero_grad()
    x, y = batch
    y_hat = model(x)
    loss = cross_entropy_with_batch(y_hat, y)
    loss.backward()
    optimizer.step()


In [ ]:
train_one_step(batch)